# Architecture Diagram


![title](../img/architecture_design.png)


## Components

- API Client: The external interface that users interact with to access the wine classification service:

  - Makes HTTP requests to the API gateway
  - Handles responses and potential errors
  - Could be implemented as a web/mobile app or programmatic client
  - Supports different types of requests (e.g. single predictions or batch processing)

- API Gateway: Serves as the entry point for all requests, handling authentication and request validation. It provides a RESTful interface for the wine classification service:

  - Handles rate limiting to prevent service overload
  - Provides API keys for client authentication
  - Routes requests to appropriate Lambda functions

- Prediction Lambda: The main serverless function that handles real-time predictions:

  - Loads the model artifacts from S3
  - Processes incoming requests
  - Returns predictions
  - Stores prediction metadata in DynamoDB
  - Outputs custom metrics to CloudWatch

- Batch Training Lambda: a separate serverless function for model maintenance:

  - Periodically retrains the model using accumulated data
  - Updates model artifacts in S3
  - Tracks training metrics and performance

- S3 Bucket: Stores model artifacts and training data:

  - Model directory: Contains pipeline and label encoder .pkl files
  - Training data directory: Stores historical data for retraining
  - Version control of models through S3 versioning

- DynamoDB: Provides high-performance storage for prediction metadata:

  - Stores request details, timestamps, and prediction results
  - Enables quick retrieval of historical predictions
  - Supports analysis of model performance over time
  - Maintains audit trails for compliance purposes

- CloudWatch: Handles monitoring and logging across the entire system:

  - Tracks model performance metrics
  - Monitors API latency and errors
  - Sets up alerts for anomalies
  - Collects logs from all Lambda functions
  - Enables custom dashboards for visualization

- Custom Metrics: Business-specific measurements tracked through CloudWatch:
  - Prediction confidence scores
  - Request volumes and patterns
  - Model accuracy trends
  - Response time distributions
  - Error rates by wine variety or price range


### **Component Interactions Overview**

The wine classification service architecture leverages AWS serverless services for scalability and maintainability. The process begins when an API Client sends a request to the system via the API Gateway, which handles authentication, authorization, and request validation. The API Gateway forwards valid requests to the Prediction Lambda, which retrieves the latest machine-learning model artifacts from the S3 Bucket. The Prediction Lambda runs the wine classification pipeline using the retrieved model and stores prediction results and metadata in DynamoDB. All operational logs and performance metrics from the Prediction Lambda and DynamoDB interactions are collected by CloudWatch, which also generates Custom Metrics for system monitoring and analysis.
Simultaneously, the Batch Training Lambda runs at scheduled intervals, accessing training data from the S3 Bucket, retraining the model, and saving updated model artifacts back to S3. CloudWatch also monitors the Batch Training Lambda, ensuring the reliability and performance of both prediction and training processes.


### **Architectural Decisions and Component Selection**

API Gateway provides a secure and scalable entry point for all client requests. AWS Lambda functions enable on-demand, serverless computation, scaling automatically with usage. S3 Bucket serves as the central storage for both training data and versioned model artifacts. DynamoDB ensures fast, reliable access to prediction metadata with its low-latency NoSQL design. CloudWatch delivers comprehensive monitoring and logging, including Custom Metrics that track key system health indicators such as request latency, model performance, and training job outcomes.


### **Production Deployment Strategy**

We adopt a modern DevOps approach using Docker for consistent application packaging across environments and AWS CDK for Infrastructure as Code, ensuring version control and reproducibility. AWS CodePipeline automates our CI/CD process, including building, testing, and deploying application and infrastructure changes. A blue-green deployment strategy minimizes downtime by running new and existing versions in parallel, allowing seamless rollbacks if necessary. This deployment strategy ensures reliable, maintainable, and scalable production releases, with automation reducing the risk of manual errors.
